In [40]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from keras.layers import Dropout

from keras.callbacks import ModelCheckpoint

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from plotly.offline import init_notebook_mode, iplot
import chart_studio.plotly as py
import plotly.graph_objs as go

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

In [3]:
train = pd.read_csv('train.csv', parse_dates=['date'],index_col=['date'])

In [4]:
test = pd.read_csv('test.csv', parse_dates=['date'],index_col=['date'])

In [5]:
train['day']=train.index.day
train['month']=train.index.month
train['year']=train.index.year
train['dayofweek']=train.index.dayofweek
train['weekofyear']=train.index.isocalendar().week
train['is_weekend']=train.index.dayofweek

test['day'] = test.index.day
test['month'] = test.index.month
test['year'] = test.index.year
test['dayofweek'] = test.index.dayofweek
test['weekofyear']  = test.index.isocalendar().week
test['is_weekend'] = test.index.dayofweek 

In [6]:
train

,store,item,sales,day,month,year,dayofweek,weekofyear,is_weekend
date,,,,,,,,,
2013-01-01,1,1,13,1,1,2013,1,1,1
2013-01-02,1,1,11,2,1,2013,2,1,2
2013-01-03,1,1,14,3,1,2013,3,1,3
2013-01-04,1,1,13,4,1,2013,4,1,4
2013-01-05,1,1,10,5,1,2013,5,1,5
...,...,...,...,...,...,...,...,...,...
2017-12-27,10,50,63,27,12,2017,2,52,2
2017-12-28,10,50,59,28,12,2017,3,52,3
2017-12-29,10,50,74,29,12,2017,4,52,4


In [7]:
train_not_17 = train[train['year'] != 2017]

In [8]:
train_not_17

,store,item,sales,day,month,year,dayofweek,weekofyear,is_weekend
date,,,,,,,,,
2013-01-01,1,1,13,1,1,2013,1,1,1
2013-01-02,1,1,11,2,1,2013,2,1,2
2013-01-03,1,1,14,3,1,2013,3,1,3
2013-01-04,1,1,13,4,1,2013,4,1,4
2013-01-05,1,1,10,5,1,2013,5,1,5
...,...,...,...,...,...,...,...,...,...
2016-12-27,10,50,60,27,12,2016,1,52,1
2016-12-28,10,50,43,28,12,2016,2,52,2
2016-12-29,10,50,68,29,12,2016,3,52,3


In [9]:
Y = train_not_17['sales']
Y

date
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
              ..
2016-12-27    60
2016-12-28    43
2016-12-29    68
2016-12-30    63
2016-12-31    64
Name: sales, Length: 730500, dtype: int64

In [10]:
X = train_not_17.drop(columns='sales')

In [11]:
X.reset_index(drop=True, inplace=True)

In [12]:
X

,store,item,day,month,year,dayofweek,weekofyear,is_weekend
0,1,1,1,1,2013,1,1,1
1,1,1,2,1,2013,2,1,2
2,1,1,3,1,2013,3,1,3
3,1,1,4,1,2013,4,1,4
4,1,1,5,1,2013,5,1,5
...,...,...,...,...,...,...,...,...
730495,10,50,27,12,2016,1,52,1
730496,10,50,28,12,2016,2,52,2
730497,10,50,29,12,2016,3,52,3
730498,10,50,30,12,2016,4,52,4


In [15]:
epochs = 40
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

In [18]:
X.shape[0]

730500

In [33]:

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
NN_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               1152      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 166,017
Trainable params: 166,017
Non-trainable params: 0
_________________________________________________________________


In [36]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [37]:
NN_model.fit(X, Y, epochs=epochs, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/40
18251/18263 [============================>.] - ETA: 0s - loss: 673.2332 - mean_squared_error: 673.2332
Epoch 00001: val_loss improved from inf to 531.97943, saving model to Weights-001--531.97943.hdf5
18263/18263 [==============================] - 60s 3ms/step - loss: 673.2289 - mean_squared_error: 673.2289 - val_loss: 531.9794 - val_mean_squared_error: 531.9794
Epoch 2/40
18256/18263 [============================>.] - ETA: 0s - loss: 586.5326 - mean_squared_error: 586.5326
Epoch 00002: val_loss did not improve from 531.97943
18263/18263 [==============================] - 59s 3ms/step - loss: 586.5405 - mean_squared_error: 586.5405 - val_loss: 579.7551 - val_mean_squared_error: 579.7551
Epoch 3/40
18261/18263 [============================>.] - ETA: 0s - loss: 575.3934 - mean_squared_error: 575.3934 ETA: 3s - loss: 575.2764 - mean - ETA: 2s - los
Epoch 00003: val_loss did not improve from 531.97943
18263/18263 [==============================] - 59s 3ms/step - loss: 575.3982 -

18260/18263 [============================>.] - ETA: 0s - loss: 556.7426 - mean_squared_error: 556.7426 - ETA: 48s - loss: 543.8748 - mean_squared_er - ETA: 44s - loss: 542.8256 - mean_squared_e - E - ETA: 41s - loss: 547.3663 - mean_squared_error: 54 - ETA: 40s - loss: 548.0295 - m - ETA: 39s - loss: 550.6447 - mean_squared_error - ETA: 38s - loss: 551.1014 - mean_squ - ETA: 37s - loss: 55
Epoch 00019: val_loss did not improve from 527.75494
18263/18263 [==============================] - 67s 4ms/step - loss: 556.7498 - mean_squared_error: 556.7498 - val_loss: 550.2116 - val_mean_squared_error: 550.2116
Epoch 20/40
18262/18263 [============================>.] - ETA: 0s - loss: 574.6694 - mean_squared_error: 574.6694 - ETA: 26s - loss: 580.8830 - mean_squared_error: 580.883 - ETA: 26 - ETA - ETA: 1s - - ETA: 0s - loss: 574.6017 - mean_squared_error: 57
Epoch 00020: val_loss did not improve from 527.75494
18263/18263 [==============================] - 71s 4ms/step - loss: 574.6710 - mean_

18251/18263 [============================>.] - ETA: 0s - loss: 493.3745 - mean_squared_error: 493.3745
Epoch 00033: val_loss did not improve from 527.75494
18263/18263 [==============================] - 57s 3ms/step - loss: 493.3560 - mean_squared_error: 493.3560 - val_loss: 2697.8364 - val_mean_squared_error: 2697.8364
Epoch 34/40
18247/18263 [============================>.] - ETA: 0s - loss: 445.6175 - mean_squared_error: 445.6175
Epoch 00034: val_loss did not improve from 527.75494
18263/18263 [==============================] - 57s 3ms/step - loss: 445.6709 - mean_squared_error: 445.6709 - val_loss: 618.6020 - val_mean_squared_error: 618.6020
Epoch 35/40
18262/18263 [============================>.] - ETA: 0s - loss: 444.6136 - mean_squared_error: 444.6136 ETA: 0s - loss: 444.9189 - mean_squared
Epoch 00035: val_loss did not improve from 527.75494
18263/18263 [==============================] - 56s 3ms/step - loss: 444.6172 - mean_squared_error: 444.6172 - val_loss: 631.1260 - val_mea

In [38]:
Train_Pred= NN_model.predict(X)

In [39]:
Train_Pred

array([[12.223711],
       [12.672754],
       [13.77302 ],
       ...,
       [35.715908],
       [35.981174],
       [36.24168 ]], dtype=float32)

In [41]:
print('Train rmse:', np.sqrt(mean_squared_error(Y,Train_Pred)))
print('Train mape:', mean_absolute_error(Y,Train_Pred))

Train rmse: 21.00583035905495
Train mape: 15.434120176354472


In [42]:
train_17 = train[train['year'] == 2017]

In [43]:
X1 = train_17.drop(columns='sales')
X1.reset_index(drop=True, inplace=True)
Y1 = train_17['sales']
X1 = np.asarray(X1).astype(np.float32)

In [46]:
Pred = NN_model.predict(X1)

In [48]:
Pred

array([[21.108248 ],
       [12.276508 ],
       [12.2634735],
       ...,
       [35.959564 ],
       [36.220036 ],
       [36.480556 ]], dtype=float32)

In [47]:
print('Validation rmse:', np.sqrt(mean_squared_error(Y1,Pred)))

Validation rmse: 25.776435286501396


In [60]:
print('Validation mape:', mean_absolute_error(Y1,Pred))

Validation mape: 18.673809258390452


In [50]:
MAPE_2_DNN = mean_absolute_error(Y,Train_Pred)

In [51]:
MAPE_2_DNN

15.434120176354472

In [58]:
type(Y1)

pandas.core.series.Series

In [62]:
pred = Pred.ravel().tolist()

In [63]:
pred

[21.108247756958008,
 12.276508331298828,
 12.263473510742188,
 12.57461929321289,
 13.762628555297852,
 14.296388626098633,
 15.646514892578125,
 16.99663734436035,
 12.260257720947266,
 12.247224807739258,
 12.606180191040039,
 13.700973510742188,
 14.412607192993164,
 15.762750625610352,
 17.112878799438477,
 12.243999481201172,
 12.230960845947266,
 12.637750625610352,
 13.63224983215332,
 14.528852462768555,
 15.878973007202148,
 17.229103088378906,
 12.22774887084961,
 12.214710235595703,
 12.669309616088867,
 13.563531875610352,
 14.64508056640625,
 15.995218276977539,
 17.345338821411133,
 12.211490631103516,
 12.198463439941406,
 14.360635757446289,
 15.575239181518555,
 16.925371170043945,
 18.275503158569336,
 19.555665969848633,
 12.881595611572266,
 14.06959342956543,
 14.341344833374023,
 15.69148063659668,
 17.041606903076172,
 18.391738891601562,
 19.73283576965332,
 12.913167953491211,
 14.01413345336914,
 14.457578659057617,
 15.807710647583008,
 17.15784454345703,
 1

In [64]:

def mean_absolute_percentage_error(y_true, y_pred): 

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [65]:
mean_absolute_percentage_error(Y1,pred)

32.33712507393828

In [68]:
train_pred = Train_Pred.ravel().tolist()

In [75]:
train_pred

[12.223711013793945,
 12.672754287719727,
 13.773019790649414,
 14.387456893920898,
 15.737590789794922,
 17.087717056274414,
 12.220491409301758,
 12.207452774047852,
 12.704320907592773,
 13.704307556152344,
 14.503694534301758,
 15.853822708129883,
 17.203947067260742,
 12.204235076904297,
 12.191198348999023,
 12.735883712768555,
 13.635576248168945,
 14.619916915893555,
 15.970056533813477,
 17.320175170898438,
 12.187982559204102,
 12.174945831298828,
 12.767454147338867,
 13.566854476928711,
 14.73615837097168,
 16.086280822753906,
 17.43640899658203,
 12.171730041503906,
 12.158693313598633,
 12.799013137817383,
 13.50225830078125,
 17.016454696655273,
 18.366573333740234,
 19.664018630981445,
 12.979740142822266,
 14.086181640625,
 14.432424545288086,
 15.782546997070312,
 17.1326847076416,
 18.482812881469727,
 19.84119415283203,
 13.011301040649414,
 14.017454147338867,
 14.548648834228516,
 15.898784637451172,
 17.248910903930664,
 18.599040985107422,
 20.018356323242188,
 

In [76]:
Y

date
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
              ..
2016-12-27    60
2016-12-28    43
2016-12-29    68
2016-12-30    63
2016-12-31    64
Name: sales, Length: 730500, dtype: int64

In [70]:
mean_absolute_percentage_error(Y,train_pred)

inf

In [79]:
NN_model.save("DNN_Model")

INFO:tensorflow:Assets written to: DNN_Model\assets
